In [1]:
# coding=utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import json
import re
import codecs

import csv
import os
import logging
import argparse
import random
from tqdm import tqdm, trange

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from run_classifier import DataProcessor,StsProcessor,convert_examples_to_features,InputExample,InputFeatures

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
#下列两个是基础类。在method里头用的
class InputExample(object):

    def __init__(self, unique_id, text_a, text_b):
        self.unique_id = unique_id#用readexamples之后会分成一个样本列表，这是用来标记第几句的
        self.text_a = text_a
        self.text_b = text_b


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
        self.unique_id = unique_id
        self.tokens = tokens#分好词加好标记的原文
        self.input_ids = input_ids#分好词加好标记的原文转数字版
        self.input_mask = input_mask#变长序列加pad填充
        self.input_type_ids = input_type_ids#分sentenceA和B的标记


def convert_examples_to_features(examples, seq_length, tokenizer): #把样本列表输入这个函数获取特征
    """Loads a data file into a list of `InputBatch`s."""
    #seq_length指max length
    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > seq_length - 2:
                tokens_a = tokens_a[0:(seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        input_type_ids = []
        tokens.append("[CLS]")
        input_type_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            input_type_ids.append(0)
        tokens.append("[SEP]")
        input_type_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                input_type_ids.append(1)
            tokens.append("[SEP]")
            input_type_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < seq_length:
            input_ids.append(0)
            input_mask.append(0)
            input_type_ids.append(0)

        assert len(input_ids) == seq_length
        assert len(input_mask) == seq_length
        assert len(input_type_ids) == seq_length

        features.append(
            InputFeatures(
                unique_id=example.unique_id,
                tokens=tokens,
                input_ids=input_ids,
                input_mask=input_mask,
                input_type_ids=input_type_ids))
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def read_examples(input_file):#这个函数用来把inputfile里的样本都改造成InputExample类
    """Read a list of `InputExample`s from an input file."""
    examples = []
    unique_id = 0
    with open(input_file, "r", encoding='utf-8') as reader:
        while True:
            line = reader.readline()
            if not line:
                break
            line = line.strip()
            text_a = None
            text_b = None
            m = re.match(r"^(.*) \|\|\| (.*)$", line)
            if m is None:
                text_a = line
            else:
                text_a = m.group(1)
                text_b = m.group(2)
            examples.append(
                InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
            unique_id += 1
    return examples

def read_exam_examples(input_file):#这个函数用来把inputfile里的样本都改造成InputExample类
    #读日本高考タイトル付与题目作为样本
    examples = []
    para=[]
    answer=[]
    with open(input_file, "r", encoding='utf-8') as reader:
        for i in range(8):
            if i <4:
                line = reader.readline()
                line = line.strip()
                para.append(line)
            else:
                line = reader.readline()
                line = line.strip()
                answer.append(line)
    for i in range(4):
        for j in range(4):        
            unique_id = 4*i+j
            text_a=para[i]
            text_b=answer[j]
            examples.append(InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
    return examples

In [3]:
exam=read_exam_examples('exam2014.txt');
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_features=convert_examples_to_features(exam, 152, tokenizer)

input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
segment_ids = torch.tensor([f.input_type_ids for f in train_features], dtype=torch.long)



model_state_dict = torch.load('/home/ying/GLUE/glue_data/STS-B/savemodel/pytorch_model.bin')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', state_dict=model_state_dict, num_labels=1)
model.cuda()

#model.eval()
   
input_ids = input_ids.cuda()
input_mask = input_mask.cuda()
segment_ids = segment_ids.cuda()
#with torch.no_grad():    
logits = model(input_ids, segment_ids, input_mask)#去掉label_ids
logits =torch.squeeze(logits)


01/08/2019 01:36:30 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ying/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
01/08/2019 01:36:33 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/ying/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
01/08/2019 01:36:33 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /home/ying/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmphgp2mhg3
01/08/2019 01:36:

In [4]:
logits=logits.detach().cpu().numpy()
#print(logits)
for i in range(4):
    for j in range(4):
        print(logits[4*i+j])
    print('/t')

2.0889711
1.7507112
1.9787178
1.846347
/t
2.42015
2.2237647
2.579243
2.5743413
/t
2.261991
1.6553184
2.0358338
1.8116326
/t
1.8486667
2.0590951
1.574862
0.6473282
/t
